In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import keras
import keras.layers as L
from keras.datasets import fashion_mnist
from keras.applications import DenseNet121
from keras.applications import VGG16

In [2]:
def show(img):
    plt.imshow(img)

In [3]:
def create_exemplar_set(size, images, trained_model):
    #'images' has images all of class=y
    exemplars=[]
    for i in images:
        exemplars.append(calc_feature_vector(i, trained_model, layer_index))
    chosen_exemplars=exemplars[:size]
    mean_exemplar=sum(exemplars)/len(images)
    return chosen_exemplars, mean_exemplar

In [4]:
def create_custom_loss():
    gan_vs_nogan=keras.losses.BinaryCrossentropy(label_smoothing=0.1)
    type_of_gan=keras.losses.CategoricalCrossentropy(label_smoothing=0.1)
    combined_loss=tf.add(gan_vs_nogan, type_of_gan)
    return combined_loss

 For multi-task learning, branching NN out
- one branch predicts the class of FashionMNIST object (classification)
- other branch predicts whether the object is a "top" or not (0/1 output)

In [5]:
def create_model(n_classes, input_dim):
    '''
        Creating categorical classification model
    '''
    
    inputs=L.Input((input_dim,input_dim,1))
    
    x=L.Conv2D(64, kernel_size=3, padding='same', strides=1)(inputs)
    x=L.LeakyReLU(0.2)(x)
    x=L.Conv2D(64, kernel_size=3, padding='same', strides=1)(x)
    x=L.LeakyReLU(0.2)(x)
    x=L.MaxPool2D(pool_size=2, strides=2)(x)
    
    shared=L.Conv2D(32, kernel_size=3, padding='same', strides=1)(x)
    x=L.LeakyReLU(0.2)(shared)
    x=L.Conv2D(32, kernel_size=3, padding='same', strides=1)(x)
    x=L.LeakyReLU(0.2)(x)
    x=L.MaxPool2D(pool_size=2, strides=2)(x)
    
    x=L.Conv2D(16, kernel_size=3, padding='same', strides=1)(x)
    x=L.LeakyReLU(0.2)(x)
    x=L.Conv2D(16, kernel_size=3, padding='same', strides=1)(x)
    x=L.LeakyReLU(0.2)(x)
    x=L.MaxPool2D(pool_size=2, strides=2)(x)
    
    x=L.Flatten()(x)
    x=L.Dense(n_classes+1)(x)
    outputs=L.Softmax()(x)
    
    model_cla=keras.Model(inputs=inputs, outputs=outputs)
    model_cla.compile(optimizer=keras.optimizers.Adam(0.02), 
                  loss=keras.losses.CategoricalCrossentropy(), 
                  metrics=['accuracy'])
    
    '''
        Creating binary classification model (a top/not a top)
    
    
    ip= L.Input(shape=shared.shape.as_list()[1:])
    x=L.LeakyReLU(0.2)(ip)
    x=L.Conv2D(32, kernel_size=3, padding='same', strides=1)(x)
    x=L.LeakyReLU(0.2)(x)
    x=L.MaxPool2D(pool_size=2, strides=2)(x)
    
    x=L.Conv2D(16, kernel_size=3, padding='same', strides=1)(x)
    x=L.LeakyReLU(0.2)(x)
    x=L.Conv2D(16, kernel_size=3, padding='same', strides=1)(x)
    x=L.LeakyReLU(0.2)(x)
    x=L.MaxPool2D(pool_size=2, strides=2)(x)
    
    x=L.Flatten()(x)
    op=L.Dense(1, activation=keras.activations.sigmoid)(x)

    model_bin=keras.Model(inputs=ip, outputs=op)
    model_bin.compile(optimizer=keras.optimizers.Adam(0.02), 
                      loss=keras.losses.BinaryCrossentropy(), 
                      metrics=['accuracy'])
    '''
    return model_cla

In [ ]:
modelc.fit(x_train, y_train, epochs=5)

In [7]:
def train_model(model, n_epochs, x_train, y_train):
    model.fit(x=x_train, 
              y=y_train, 
              epochs=n_epochs)
    return model

In [8]:
def calc_feature_vector(image, model, layer_index):
    #extract feature vector from a layer of ALREADY-TRAINED model
    extractor=keras.Model(model.inputs(), [model.layers[layer_index].output])
    vector=extractor(image)
    #divide vector by norm
    return vector

In [9]:
def classify(X, mean_exemplar_sets):    
    image_vector=calc_feature_vector(X)
    #exemplar set closest to X feature map is chosen
    return None

In [90]:
def transform_outputs(y_train, y_test):
    labeldict={}
    labeldict[0]=1
    labeldict[1]=0
    labeldict[2]=1
    labeldict[3]=1
    labeldict[4]=1
    labeldict[5]=0
    labeldict[6]=1
    labeldict[7]=0
    labeldict[8]=0
    labeldict[9]=0
    
    y_train_ints=y_train
    y_test_ints=y_test
    y_train=keras.utils.to_categorical(y_train)
    y_test=keras.utils.to_categorical(y_test)
    y_trainz=np.ndarray((0,11))
    y_testz=np.ndarray((0,11))
    
    for i in range(len(y_train)):
        temp=np.append(y_train[i], labeldict[y_train_ints[i]])
        y_trainz=np.append(y_trainz, [temp], axis=0)
        
    for i in range(len(y_test)):
        temp=np.append(y_test[i], labeldict[y_test_ints[i]])
        y_testz=np.append(y_testz, [temp], axis=0)
    return y_trainz, y_testz

In [91]:
#def main():
(x_train, y_train), (x_test, y_test)=tf.keras.datasets.fashion_mnist.load_data()

In [92]:
x_train=x_train.reshape((x_train.shape[0], 28,28,1))
x_test=x_test.reshape((x_test.shape[0],28,28,1))

Cell below is very heavy; already run and exported as csv

In [93]:
#y_train, y_test=transform_outputs(y_train, y_test)

In [94]:
#np.savetxt("ytrain_transformed.csv", y_train, delimiter=',')
#np.savetxt("ytest_transformed.csv", y_test, delimiter=',')

In [98]:
y_tr=np.loadtxt(open("ytrain_transformed.csv", "rb"), delimiter=',')
y_te=np.loadtxt(open("ytest_transformed.csv", "rb"), delimiter=',')

In [95]:
model=create_model(10,28)

In [96]:
y_train.shape

(60000, 11)

In [97]:
train_model(model, 5, x_train, y_train)

Epoch 1/5
 490/1875 [======>.......................] - ETA: 2:57 - loss: nan - accuracy: 0.1134

KeyboardInterrupt: 